# Overview of project
We are provided historic data of raw material deliveries and orders through the end of 2024. GOAL: Develop a model that forecasts the cumulative weight of incoming deliveries of each raw material from Jan 1, 2025, up to any specified end date between Jan 1 and May 31, 2025.

- recievals = historical records of material recievals
- purchase_orders = ordered quantities and expected deliv
- materials(opt) = metadata on various raw materials
- transportation(opt) = transport-related data

QuantileLoss0.2(Fi,Ai) = max(0.2*(Ai − Fi), 0.8*(Fi − Ai)).

rm_id = unique identifer for raw material

In [65]:
# We need to explore the data

# First I want to check the difference between purchase orders
# and recievals. How much was the difference between the two?

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
data_orders = pd.read_csv('data/kernel/purchase_orders.csv')
data_receivals = pd.read_csv('data/kernel/receivals.csv')

# Link the 5 first data orders to the recievals
data = pd.merge(data_orders.head(5), data_receivals, on='purchase_order_id', suffixes=('_order', '_receival'), how='left')

data.head(2)

# NOT EVERY ORDER HAS A RECEIVAL? Oh... it makes sense cause some orders are never received? But I put the 5 in the head.... 5....
# 5 whole orders are not received? That seems like a lot.... Nah maybe it's just purchase_order_id is a bad key to merge on.
# Let's try purchase_order_item_no... # Absolutely not. I forgot it was like simple 1 etc....

,purchase_order_id,purchase_order_item_no_order,quantity,delivery_date,product_id_order,product_version,created_date_time,modified_date_time,unit_id,unit,...,status,rm_id,product_id_receival,purchase_order_item_no_receival,receival_item_no,batch_id,date_arrival,receival_status,net_weight,supplier_id
0,1,1,-14.0,2003-05-12 00:00:00.0000000 +02:00,91900143,1,2003-05-12 10:00:48.0000000 +00:00,2004-06-15 06:16:18.0000000 +00:00,NaN,NaN,...,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22,1,23880.0,2003-05-27 00:00:00.0000000 +02:00,91900160,1,2003-05-27 12:42:07.0000000 +00:00,2012-06-29 09:41:13.0000000 +00:00,NaN,NaN,...,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Count all rows with quantity ordered negative
print(data_orders['quantity'].lt(0).sum())

print(data_orders['quantity'].eq(150000).sum())
# 6 rows with negative quantity... prob wrong..

6
563


In [61]:
# I want to check the transportation of the 5 orders in the head
data_transport = pd.read_csv('data/extended/transportation.csv')

data = pd.merge(data_orders.head(5), data_transport, on='purchase_order_id', suffixes=('_order', '_transport'))

print(data)

# Observation: Not all orders are transported either....

Empty DataFrame
Columns: [purchase_order_id, purchase_order_item_no_order, quantity, delivery_date, product_id_order, product_version, created_date_time, modified_date_time, unit_id, unit, status_id, status, rm_id, product_id_transport, purchase_order_item_no_transport, receival_item_no, batch_id, transporter_name, vehicle_no, unit_status, vehicle_start_weight, vehicle_end_weight, gross_weight, tare_weight, net_weight, wood, ironbands, plastic, water, ice, other, chips, packaging, cardboard]
Index: []

[0 rows x 34 columns]


In [53]:
# I want to check the material details of the 5 orders in the head
# NVM... cooked... orders have nothing to directly link to materials

In [ ]:
# OKAY! Let's try to drop all the orders with no recievals maybe? And try to predict? But in a real scenario I probably shouldn't
# Cause maybe the orders with no recievals are equal to 0 recieved? But I don't know if that's true. Gotta test
# So try 2 stuff: 1. drop the orders with no recievals, 2. set the recievals to 0 if no recievals

# But first I neeed to know what my model will predict? Like will I get orders and recievals? Or just predict by the order prev?
# Okay I don't think I'll get more orders in the future, so I guess I just have to predict based on previous orders

# Purchase orders have an expected delivery_date though.
# They are using YYYY-MM-DD format I guess
# They want from 2025-01-01 to 2025-05-31
# We got some deliveries expected in 2025-03-XX, but none after, so we prob need to predict that there will be more orders.

# By making a model that predicts the quantity ordered based on previous orders, I can then use that to predict future orders
# I should prob make a model for each of the materials and then sum them up for each order date

In [62]:
# Starting by dropping the orders with no recievals
data = pd.merge(
    data_orders,
    data_receivals,
    on=['purchase_order_id', 'purchase_order_item_no'],
    suffixes=('_order', '_receival')
)

# 122537 rows, but recievals has 122590 rows. So some recievals are from orders not in the orders dataset?
data_extra_receivals = pd.merge(
    data_orders,
    data_receivals,
    on=['purchase_order_id', 'purchase_order_item_no'],
    suffixes=('_order', '_receival'),
    how='right'
)

print(data_extra_receivals.shape)
print(data.shape)
# 122591 rows, so 54 extra recievals that are not in the orders dataset
# Let's check if they are all from the same purchase_order_id

# I want the data_extra_receivals rows that are not in data
data_diff = pd.concat([data_extra_receivals, data]).drop_duplicates(keep=False)
print(data_diff.head(5))



(122590, 20)
(122537, 20)
       purchase_order_id  purchase_order_item_no  quantity delivery_date  \
61798                NaN                     NaN       NaN           NaN   
63356                NaN                     NaN       NaN           NaN   
64105                NaN                     NaN       NaN           NaN   
65448                NaN                     NaN       NaN           NaN   
71981                NaN                     NaN       NaN           NaN   

       product_id_order  product_version created_date_time modified_date_time  \
61798               NaN              NaN               NaN                NaN   
63356               NaN              NaN               NaN                NaN   
64105               NaN              NaN               NaN                NaN   
65448               NaN              NaN               NaN                NaN   
71981               NaN              NaN               NaN                NaN   

       unit_id unit  status_id

In [63]:
# Check recievals with no purchase order id or purchase order item no
print(data_receivals['purchase_order_id'].isna().sum())
print(data_receivals['purchase_order_item_no'].isna().sum())

53
53


In [67]:
# Okay let me first try using the data with recievals and where the recievals can be linked to purchase
data = pd.merge(
    data_orders,
    data_receivals,
    on=['purchase_order_id', 'purchase_order_item_no'],
    suffixes=('_order', '_receival'),
    how='left'
)

In [82]:
data = pd.read_csv('data/kernel/receivals.csv')
print(data['receival_status'].unique())
print(data['receival_status'].value_counts())

['Completed' 'Finished unloading' 'Planned' 'Start unloading']
receival_status
Completed             122448
Finished unloading       106
Start unloading           32
Planned                    4
Name: count, dtype: int64


In [1]:
# Found out that some materials cease to be ordered after some time. Maybe they are obsolete?
# Some dates use different time zones than others
# Some units are in KG, LBs and pounds --> Need to make them to KG and drop that column
# Some materials stock are deleted in the materials file

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

############## CLEANING THE PURCHASE ORDERS DATA ##############

orders = pd.read_csv("./data/kernel/purchase_orders.csv")

# Make the orders with PUND in KGs, and change quantity accordingly
# 1 PUND = 0,45359237 kilogram
orders.loc[orders['unit'] == 'PUND', 'quantity'] = orders.loc[orders['unit'] == 'PUND', 'quantity'] * 0.45359237
# Change the unit to KG too: orders.loc[orders['unit'] == 'PUND', 'unit'] = 'KG'
# Drop unit_id and unit columns
orders = orders.drop(columns=['unit_id', 'unit'])

# Time is in GMT+2 which is Norway time
# Make delivery_date, created_date_time and modified_date_time to GMT +2
orders['delivery_date'] = pd.to_datetime(orders['delivery_date'], utc=True).dt.tz_convert('Etc/GMT-2')
orders['created_date_time'] = pd.to_datetime(orders['created_date_time'], utc=True).dt.tz_convert('Etc/GMT-2')
orders['modified_date_time'] = pd.to_datetime(orders['modified_date_time'], utc=True).dt.tz_convert('Etc/GMT-2')

# Save the cleaned data to a new CSV file in data_cleaned folder
#orders.to_csv('./data_cleaned/purchase_orders_cleaned.csv', index=False)

   purchase_order_id  purchase_order_item_no  quantity  \
0                  1                       1     -14.0   
1                 22                       1   23880.0   

              delivery_date  product_id  product_version  \
0 2003-05-12 00:00:00+02:00    91900143                1   
1 2003-05-27 00:00:00+02:00    91900160                1   

          created_date_time        modified_date_time  status_id  status  
0 2003-05-12 12:00:48+02:00 2004-06-15 08:16:18+02:00          2  Closed  
1 2003-05-27 14:42:07+02:00 2012-06-29 11:41:13+02:00          2  Closed  


In [24]:
### CLEANING THE RECEIVALS DATA ###
receivals = pd.read_csv("./data/kernel/receivals.csv")

# Make the date_arrival to GMT +2
receivals['date_arrival'] = pd.to_datetime(receivals['date_arrival'], utc=True).dt.tz_convert('Etc/GMT-2')
# Save the cleaned data to a new CSV file in data_cleaned folder
receivals.to_csv('./data_cleaned/receivals_cleaned.csv', index=False)

In [52]:
# Method 2: Merge orders and receivals directly, then aggregate the recievals per order line
# This will create duplicate rows for orders with multiple recievals, but we can aggregate them

# --- Load data ---
orders = pd.read_csv(
    "./data_cleaned/purchase_orders_cleaned.csv",
    parse_dates=["delivery_date", "created_date_time", "modified_date_time"]
)
receivals = pd.read_csv(
    "./data_cleaned/receivals_cleaned.csv",
    parse_dates=["date_arrival"]
)

# --- Merge orders and receivals WITHOUT aggregation ---
orders_with_receivals = orders.merge(
    receivals,
    on=["purchase_order_id", "purchase_order_item_no"],
    how="left",
    suffixes=('_order', '_receival')
)

# --- Fill missing values for orders with no receivals ---
orders_with_receivals["net_weight"] = orders_with_receivals["net_weight"].fillna(0)
orders_with_receivals["date_arrival"] = pd.to_datetime(orders_with_receivals["date_arrival"])

# --- Derived features ---
orders_with_receivals["fill_fraction"] = orders_with_receivals["net_weight"] / orders_with_receivals["quantity"]
orders_with_receivals["lead_time"] = (
    orders_with_receivals["date_arrival"] - orders_with_receivals["delivery_date"]
).dt.days
orders_with_receivals["lead_time"] = orders_with_receivals["lead_time"].fillna(0)

# --- Save result ---
#orders_with_receivals.to_csv("./data_cleaned/orders_with_receivals_detailed.csv", index=False)

print(orders_with_receivals.shape)


(133409, 20)


In [77]:
orders_merged = pd.read_csv("./data_cleaned/orders_with_receivals.csv")

# print how many orders have value for total_received_qty that is different from 0
print((orders_merged['total_received_qty'] != 0).sum())
print((orders_merged['total_received_qty'] == 0).sum())
print(orders_merged.shape)

27410
10887
(38297, 15)


In [55]:
# Check how many orders with no receivals
print((orders_with_receivals['net_weight'] == 0).sum())
print(orders_with_receivals.shape)
# 122 591 rows, so 54 extra recievals that are not in the orders dataset
# 11 026 orders with no receivals
# 122 537 + 11 026 = 133 563
# So it doesn't make sense that I get 133 409 rows when merging
# Errr... whatever for now I guess
# Ehhrm why do I get 10 887 above and 11 026 here? errrm...

# TODO: FIND OUT WHY THE DIFFERENCE IN ROWS WHEN MERGING

11026
(133409, 20)


In [3]:
# Okay now use orders_with_receivals_detailed.csv to make a model that predicts net_weight based on previous orders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

orders = pd.read_csv("./data_cleaned/purchase_orders_cleaned.csv", parse_dates=["delivery_date", "created_date_time", "modified_date_time"])
receivals = pd.read_csv("./data_cleaned/receivals_cleaned.csv", parse_dates=["date_arrival"])
orders_with_receivals = pd.read_csv("./data_cleaned/orders_with_receivals_detailed.csv", parse_dates=["delivery_date", "created_date_time", "modified_date_time", "date_arrival"])

# Make sure dates are tz-naive for calculations
orders_with_receivals['date_arrival'] = orders_with_receivals['date_arrival'].dt.tz_convert(None)
orders_with_receivals['delivery_date'] = orders_with_receivals['delivery_date'].dt.tz_convert(None)
orders_with_receivals['created_date_time'] = orders_with_receivals['created_date_time'].dt.tz_convert(None)
orders_with_receivals['modified_date_time'] = orders_with_receivals['modified_date_time'].dt.tz_convert(None)

# Mean fill_fraction, lead_time and weekly order quantity per material

material_stats = orders_with_receivals.groupby('rm_id').agg(
    avg_fill_fraction=('fill_fraction', 'mean'),
    avg_lead_time=('lead_time', 'mean'),
    avg_weekly_order_qty=('quantity', lambda x: x.sum() / ((orders_with_receivals['delivery_date'].max() - orders_with_receivals['delivery_date'].min()).days / 7))
).reset_index()

print(material_stats.head(2))

   rm_id  avg_fill_fraction  avg_lead_time  avg_weekly_order_qty
0  342.0           0.479615           23.0             42.573099
1  343.0           0.004804         -642.0           3708.771930


In [4]:
# But when I think closely. Why don't I make a model that trains on
# data up to 2013 and predicts 2014? Cause then I can actually see if it works
# So I need to split the data into train and test based on date

# Sidenote: I just thought about something for transportation: Mby some transporter names are more reliable than others?
# Like some transporters always deliver on time, while others are late. They prob get better as time goes on too?

# Okay let's start with splitting the data into train and test based on date
train_data = orders_with_receivals[orders_with_receivals['delivery_date'] < '2024-01-01']
test_data = orders_with_receivals[orders_with_receivals['delivery_date'] >= '2024-01-01']

# The thing is I have so much data from previous years so I feel like I will overfit if I use all of it
# So I will use only the most recent 3 years of data for training? Hmm some years we got events like
# COVID or NM in skiing that might make some years different too

train_data = train_data[train_data['delivery_date'] >= '2021-01-01']

print(train_data.shape)
print(test_data.shape)




(19430, 20)
(6340, 20)
